In [ ]:
# Importing Pandas to read CSVs
import pandas as pd

In [ ]:
# Defining variables for datasets' url's
url_csv_user = "https://drive.google.com/uc?id=12tDOdNR-bDXUx4xRIXvau2PbDzE_AAkM"
url_csv_transaction = "https://drive.google.com/uc?id=12dGc5ub-VN8i0A8-gCB0NH7exMJONCF2"
url_csv_products = "https://drive.google.com/uc?id=12myq3QdveeE4F4Wm8pZ18RVgu07l2m_P"


In [ ]:
# Reading and Analyzing CSVs - 1/3 - User Dataset
df_user = pd.read_csv(url_csv_user)
print(df_user.info())

# I see that we have a lot of rows with null values for birth date, state, language and gender.
# I don't think none of them are essential, so I'll keep the users that have missing only SOME
# of these information. But I'll check for rows without any of the information and delete them

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   ID            100000 non-null  object
 1   CREATED_DATE  100000 non-null  object
 2   BIRTH_DATE    96325 non-null   object
 3   STATE         95188 non-null   object
 4   LANGUAGE      69492 non-null   object
 5   GENDER        94108 non-null   object
dtypes: object(6)
memory usage: 4.6+ MB
None


In [ ]:
# Checking for rows where all of BIRTH_DATE, STATE, LANGUAGE and GENDER are null

subset_null_values = df_user.iloc[:, 2:5]
null_rows = df_user[subset_null_values.isnull().all(axis=1)]
print(null_rows.info())
# 29 users had none of the information. I'll delete them for our analysis, but
# they must keep on the database for later enrichment using CRM campaigns or
# something like that.

<class 'pandas.core.frame.DataFrame'>
Index: 29 entries, 428 to 96720
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   ID            29 non-null     object
 1   CREATED_DATE  29 non-null     object
 2   BIRTH_DATE    0 non-null      object
 3   STATE         0 non-null      object
 4   LANGUAGE      0 non-null      object
 5   GENDER        0 non-null      object
dtypes: object(6)
memory usage: 1.6+ KB
None


In [ ]:
# Removing users with no information
df_user_clean = df_user.dropna(subset=['BIRTH_DATE', 'STATE', 'LANGUAGE', 'GENDER'], how='all')

# Checking how many rows remained
print(f"Before: {len(df_user)} users")
print(f"After: {len(df_user_clean)} users")


Before: 100000 users
After: 99971 users


In [ ]:
# The column BIRTH_DATE is showing time information, i'll convert to date only

df_user_clean['BIRTH_DATE'] = pd.to_datetime(df_user_clean['BIRTH_DATE'])
df_user_clean['BIRTH_DATE'] = df_user_clean['BIRTH_DATE'].dt.date

print(df_user_clean.head())


                         ID               CREATED_DATE  BIRTH_DATE STATE  \
0  5ef3b4f17053ab141787697d  2020-06-24 20:17:54.000 Z  2000-08-11    CA   
1  5ff220d383fcfc12622b96bc  2021-01-03 19:53:55.000 Z  2001-09-24    PA   
2  6477950aa55bb77a0e27ee10  2023-05-31 18:42:18.000 Z  1994-10-28    FL   
3  658a306e99b40f103b63ccf8  2023-12-26 01:46:22.000 Z         NaT    NC   
4  653cf5d6a225ea102b7ecdc2  2023-10-28 11:51:50.000 Z  1972-03-19    PA   

  LANGUAGE  GENDER  
0   es-419  female  
1       en  female  
2   es-419  female  
3       en     NaN  
4       en  female  


<ipython-input-35-965424567182>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_clean['BIRTH_DATE'] = pd.to_datetime(df_user_clean['BIRTH_DATE'])
<ipython-input-35-965424567182>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_clean['BIRTH_DATE'] = df_user_clean['BIRTH_DATE'].dt.date


In [ ]:
# Spanish language is also showing some weird code. Cleaning it and also making
# the language names clearer
df_user_clean['LANGUAGE'] = df_user_clean['LANGUAGE'].replace('es-419', 'Spanish')
df_user_clean['LANGUAGE'] = df_user_clean['LANGUAGE'].replace('en', 'English')
print(df_user_clean['LANGUAGE'].unique())

['Spanish' 'English' nan]


<ipython-input-36-5243bb191152>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_clean['LANGUAGE'] = df_user_clean['LANGUAGE'].replace('es-419', 'Spanish')
<ipython-input-36-5243bb191152>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_clean['LANGUAGE'] = df_user_clean['LANGUAGE'].replace('en', 'English')


In [ ]:
# Removing this "Z's" and the "000's" from CREATED_DATE
df_user_clean['CREATED_DATE'] = df_user_clean['CREATED_DATE'].str.replace('Z', '')
df_user_clean['CREATED_DATE'] = df_user_clean['CREATED_DATE'].str.replace('.000', '')
print(df_user_clean.head())

                         ID          CREATED_DATE  BIRTH_DATE STATE LANGUAGE  \
0  5ef3b4f17053ab141787697d  2020-06-24 20:17:54   2000-08-11    CA  Spanish   
1  5ff220d383fcfc12622b96bc  2021-01-03 19:53:55   2001-09-24    PA  English   
2  6477950aa55bb77a0e27ee10  2023-05-31 18:42:18   1994-10-28    FL  Spanish   
3  658a306e99b40f103b63ccf8  2023-12-26 01:46:22          NaT    NC  English   
4  653cf5d6a225ea102b7ecdc2  2023-10-28 11:51:50   1972-03-19    PA  English   

   GENDER  
0  female  
1  female  
2  female  
3     NaN  
4  female  


<ipython-input-37-2cb6678ac622>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_clean['CREATED_DATE'] = df_user_clean['CREATED_DATE'].str.replace('Z', '')
<ipython-input-37-2cb6678ac622>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_user_clean['CREATED_DATE'] = df_user_clean['CREATED_DATE'].str.replace('.000', '')


In [ ]:
# Ok, User dataset sounds good enough for the moment
# Reading and Analyzing CSVs - 2/3 - Transactions Dataset
df_transaction = pd.read_csv(url_csv_transaction)
print(df_transaction.info())

# Something strange about this dataset is that it contains information of BARCODE os the products sold, but this is a transactions dataset, which incorrectically indicates that there was only one product on each purchase

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   RECEIPT_ID      50000 non-null  object 
 1   PURCHASE_DATE   50000 non-null  object 
 2   SCAN_DATE       50000 non-null  object 
 3   STORE_NAME      50000 non-null  object 
 4   USER_ID         50000 non-null  object 
 5   BARCODE         44238 non-null  float64
 6   FINAL_QUANTITY  50000 non-null  object 
 7   FINAL_SALE      50000 non-null  object 
dtypes: float64(1), object(7)
memory usage: 3.1+ MB
None


In [ ]:
# Identifying if the receipt ID's are all unique
print(df_transaction['RECEIPT_ID'].nunique())

24440


In [ ]:
df_transaction_clean = df_transaction

In [ ]:
# All the receipt ID's are doubled. But before cleaninig this, We'll use the
# duplicated rows to fill the null FINAL_SALE values, using the other row with
# the same RECEIPT_ID
df_transaction_clean['FINAL_SALE'] = pd.to_numeric(df_transaction_clean['FINAL_SALE'], errors='coerce')

# Correcting also the FINAL_QUANTITY column, making the rows with info "zero" into null
df_transaction_clean['FINAL_QUANTITY'] = pd.to_numeric(df_transaction_clean['FINAL_QUANTITY'], errors='coerce')
df_transaction_clean.head(15)

df_transaction_clean['FINAL_SALE'] = df_transaction_clean['FINAL_SALE'].fillna(
    df_transaction_clean.groupby('RECEIPT_ID')['FINAL_SALE'].transform('first')
)

df_transaction_clean.sort_values(by='RECEIPT_ID', ascending=False)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
24999,fffe8012-7dcf-4d84-b6c6-feaacab5074a,2024-09-07,2024-09-08 08:21:25.648 Z,WALGREENS,5f53c62bd683c715b9991b20,7.432310e+10,NaN,2.98
25233,fffe8012-7dcf-4d84-b6c6-feaacab5074a,2024-09-07,2024-09-08 08:21:25.648 Z,WALGREENS,5f53c62bd683c715b9991b20,7.432310e+10,2.0,2.98
24998,fffbfb2a-7c1f-41c9-a5da-628fa7fcc746,2024-07-28,2024-07-28 11:47:34.180 Z,WALMART,62a0c8f7d966665570351bb8,1.300001e+10,1.0,3.48
31602,fffbfb2a-7c1f-41c9-a5da-628fa7fcc746,2024-07-28,2024-07-28 11:47:34.180 Z,WALMART,62a0c8f7d966665570351bb8,1.300001e+10,1.0,3.48
28152,fffbb112-3cc5-47c2-b014-08db2f87e0c7,2024-07-30,2024-08-04 11:43:31.474 Z,WALMART,5eb59d6be7012d13941af5e2,8.180000e+11,1.0,4.88
...,...,...,...,...,...,...,...,...
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.0,2.54
39291,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,1.0,1.49
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,NaN,1.49
41567,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.0,1.54


In [ ]:
# FiNAL_QUANTITY is with a similar problem, with most of the rows showing the
# information only for one of the two RECEIPT_IDs. Let's do the same process to
# fill the gaps.
df_transaction_clean['FINAL_QUANTITY'] = df_transaction_clean['FINAL_QUANTITY'].fillna(
    df_transaction_clean.groupby('RECEIPT_ID')['FINAL_QUANTITY'].transform('first')
)

df_transaction_clean.sort_values(by='RECEIPT_ID', ascending=False)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
24999,fffe8012-7dcf-4d84-b6c6-feaacab5074a,2024-09-07,2024-09-08 08:21:25.648 Z,WALGREENS,5f53c62bd683c715b9991b20,7.432310e+10,2.0,2.98
25233,fffe8012-7dcf-4d84-b6c6-feaacab5074a,2024-09-07,2024-09-08 08:21:25.648 Z,WALGREENS,5f53c62bd683c715b9991b20,7.432310e+10,2.0,2.98
24998,fffbfb2a-7c1f-41c9-a5da-628fa7fcc746,2024-07-28,2024-07-28 11:47:34.180 Z,WALMART,62a0c8f7d966665570351bb8,1.300001e+10,1.0,3.48
31602,fffbfb2a-7c1f-41c9-a5da-628fa7fcc746,2024-07-28,2024-07-28 11:47:34.180 Z,WALMART,62a0c8f7d966665570351bb8,1.300001e+10,1.0,3.48
28152,fffbb112-3cc5-47c2-b014-08db2f87e0c7,2024-07-30,2024-08-04 11:43:31.474 Z,WALMART,5eb59d6be7012d13941af5e2,8.180000e+11,1.0,4.88
...,...,...,...,...,...,...,...,...
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1.0,2.54
39291,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,1.0,1.49
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,1.0,1.49
41567,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1.0,1.54


In [ ]:
# FINAL_QUANTITY should be an integer number, so let's change it.
df_transaction_clean['FINAL_QUANTITY'] = df_transaction_clean['FINAL_QUANTITY'].astype(int)

df_transaction_clean.sort_values(by='FINAL_QUANTITY', ascending=False)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
24810,fe0780d1-2d02-4822-8f12-7056b1814f17,2024-08-09,2024-08-11 17:52:18.523 Z,MAIN STREET MARKET,5d197f9dd08976510c49d0e6,4.800135e+10,276,5.89
42410,fe0780d1-2d02-4822-8f12-7056b1814f17,2024-08-09,2024-08-11 17:52:18.523 Z,MAIN STREET MARKET,5d197f9dd08976510c49d0e6,4.800135e+10,276,5.89
28613,23db217d-9c47-423d-b4ea-12896b922026,2024-09-06,2024-09-06 14:03:35.904 Z,WALMART,60a5363facc00d347abadc8e,1.200500e+10,18,64.44
3538,23db217d-9c47-423d-b4ea-12896b922026,2024-09-06,2024-09-06 14:03:35.904 Z,WALMART,60a5363facc00d347abadc8e,1.200500e+10,18,64.44
37632,41ea7b47-1e2b-4bb7-bb65-86fc3d973cf7,2024-08-22,2024-08-22 12:39:24.167 Z,SHOP RITE,61f4093fa043891075334468,3.111119e+11,16,47.84
...,...,...,...,...,...,...,...,...
4905,32752629-5882-4f01-8eee-dbbdb2756448,2024-08-27,2024-08-27 21:23:31.410 Z,WHOLE FOODS MARKET,60d67d1d525ce775c13d370c,NaN,0,0.24
4001,28abe3bc-bc23-4ba1-8733-bb65ff5f958c,2024-08-04,2024-08-04 12:40:47.029 Z,THE FRESH GROCER,63e02cf0b425eb11a4752484,4.608400e+04,0,1.16
3878,277b1f93-4de0-47bf-a094-319de868e601,2024-08-26,2024-08-26 11:56:03.764 Z,MEIJER,5fa7cf2a974a6c126e73e84c,4.094500e+04,0,0.29
3188,2057913b-b444-4a4f-9935-3bc72a13f1b9,2024-07-28,2024-08-03 09:44:59.328 Z,MEIJER,652efc6ca65d634c1c298c09,NaN,0,0.41


In [ ]:
# Now we are free to remove the RECEIPT_ID duplicates
df_transaction_clean = df_transaction_clean.drop_duplicates(subset='RECEIPT_ID', keep='first')
df_transaction_clean.head(20)

,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,1.530001e+10,1,1.54
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,NaN,1,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,7.874223e+10,1,2.54
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,7.833997e+11,1,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,4.790050e+10,1,5.29
5,0002d8cd-1701-4cdd-a524-b70402e2dbc0,2024-06-24,2024-06-24 19:44:54.247 Z,WALMART,5dcc6c510040a012b8e76924,6.811314e+11,1,1.46
6,000550b2-1480-4c07-950f-ff601f242152,2024-07-06,2024-07-06 19:27:48.586 Z,WALMART,5f850bc9cf9431165f3ac175,4.920091e+10,1,3.12
7,00096c49-8b04-42f9-88ce-941c5e06c4a7,2024-08-19,2024-08-21 17:35:21.902 Z,TARGET,6144f4f1f3ef696919f54b5c,7.830007e+10,1,3.59
8,000e1d35-15e5-46c6-b6b3-33653ed3d27e,2024-08-13,2024-08-13 18:21:07.931 Z,WALMART,61a6d926f998e47aad33db66,5.200001e+10,1,0.98
9,0010d87d-1ad2-4e5e-9a25-cec736919d15,2024-08-04,2024-08-04 18:01:47.787 Z,ALDI,66686fc2e04f743a096ea808,NaN,1,2.29


In [ ]:
# The barcodes are being showed as numbers, let's change them to text
df_transaction_clean['BARCODE'] = df_transaction_clean['BARCODE'].astype(str)
# And then remove the ".0" at the end
df_transaction_clean['BARCODE'] = df_transaction_clean['BARCODE'].str.replace('.0', '')
# Making the fake "nan" values null
df_transaction_clean['BARCODE'] = df_transaction_clean['BARCODE'].replace('nan', None)
# And correcting a "-1" value for BARCODE to null
df_transaction_clean['BARCODE'] = df_transaction_clean['BARCODE'].replace('-1', None)

df_transaction_clean.head(20)

# We'll keep the blank values on BARCODE because we can have other insights even
# for transactions without this information. But then we'll have to talk to the
# responsible team why some purchases are coming without this information and
# address the correction


<ipython-input-45-24c2a52ad809>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transaction_clean['BARCODE'] = df_transaction_clean['BARCODE'].astype(str)
<ipython-input-45-24c2a52ad809>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transaction_clean['BARCODE'] = df_transaction_clean['BARCODE'].str.replace('.0', '')
<ipython-input-45-24c2a52ad809>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead



,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539 Z,WALMART,63b73a7f3d310dceeabd4758,15300014978,1,1.54
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206 Z,ALDI,62c08877baa38d1a1f6c211a,None,1,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813 Z,WALMART,60842f207ac8b7729e472020,78742229751,1,2.54
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468 Z,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,1,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549 Z,RANDALLS,6193231ae9b3d75037b0f928,47900501183,1,5.29
5,0002d8cd-1701-4cdd-a524-b70402e2dbc0,2024-06-24,2024-06-24 19:44:54.247 Z,WALMART,5dcc6c510040a012b8e76924,681131411295,1,1.46
6,000550b2-1480-4c07-950f-ff601f242152,2024-07-06,2024-07-06 19:27:48.586 Z,WALMART,5f850bc9cf9431165f3ac175,49200905548,1,3.12
7,00096c49-8b04-42f9-88ce-941c5e06c4a7,2024-08-19,2024-08-21 17:35:21.902 Z,TARGET,6144f4f1f3ef696919f54b5c,78300069942,1,3.59
8,000e1d35-15e5-46c6-b6b3-33653ed3d27e,2024-08-13,2024-08-13 18:21:07.931 Z,WALMART,61a6d926f998e47aad33db66,52000011227,1,0.98
9,0010d87d-1ad2-4e5e-9a25-cec736919d15,2024-08-04,2024-08-04 18:01:47.787 Z,ALDI,66686fc2e04f743a096ea808,None,1,2.29


In [ ]:
# Removing these "Z's" and the "000's" from CREATED_DATE
df_transaction_clean['SCAN_DATE'] = df_transaction_clean['SCAN_DATE'].str.replace('Z', '')
df_transaction_clean.head()

# And we are good to go with this dataframe. Let's check the products database

<ipython-input-46-36bf30070e97>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_transaction_clean['SCAN_DATE'] = df_transaction_clean['SCAN_DATE'].str.replace('Z', '')


,RECEIPT_ID,PURCHASE_DATE,SCAN_DATE,STORE_NAME,USER_ID,BARCODE,FINAL_QUANTITY,FINAL_SALE
0,0000d256-4041-4a3e-adc4-5623fb6e0c99,2024-08-21,2024-08-21 14:19:06.539,WALMART,63b73a7f3d310dceeabd4758,15300014978,1,1.54
1,0001455d-7a92-4a7b-a1d2-c747af1c8fd3,2024-07-20,2024-07-20 09:50:24.206,ALDI,62c08877baa38d1a1f6c211a,None,1,1.49
2,00017e0a-7851-42fb-bfab-0baa96e23586,2024-08-18,2024-08-19 15:38:56.813,WALMART,60842f207ac8b7729e472020,78742229751,1,2.54
3,000239aa-3478-453d-801e-66a82e39c8af,2024-06-18,2024-06-19 11:03:37.468,FOOD LION,63fcd7cea4f8442c3386b589,783399746536,1,3.49
4,00026b4c-dfe8-49dd-b026-4c2f0fd5c6a1,2024-07-04,2024-07-05 15:56:43.549,RANDALLS,6193231ae9b3d75037b0f928,47900501183,1,5.29


In [ ]:
# Reading and Analyzing CSVs - 3/3 - Products Dataset
df_product = pd.read_csv(url_csv_products)

In [ ]:
print(df_product.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 845552 entries, 0 to 845551
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   CATEGORY_1    845441 non-null  object 
 1   CATEGORY_2    844128 non-null  object 
 2   CATEGORY_3    784986 non-null  object 
 3   CATEGORY_4    67459 non-null   object 
 4   MANUFACTURER  619078 non-null  object 
 5   BRAND         619080 non-null  object 
 6   BARCODE       841527 non-null  float64
dtypes: float64(1), object(6)
memory usage: 45.2+ MB
None


In [ ]:
# Removing products with null BARCODE (This table is not made to be analyzed on
# its own and these products will not join other tables)
df_product_clean = df_product.dropna(subset=['BARCODE'])

In [ ]:
# Transforming BARCODE to text
df_product_clean['BARCODE'] = df_product_clean['BARCODE'].astype(str)
df_product_clean['BARCODE'] = df_product_clean['BARCODE'].str.replace('.0', '')

<ipython-input-50-e70445951bb1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_clean['BARCODE'] = df_product_clean['BARCODE'].astype(str)
<ipython-input-50-e70445951bb1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_product_clean['BARCODE'] = df_product_clean['BARCODE'].str.replace('.0', '')


In [ ]:
# Checking if there is rows entirely with null values excepting BARCODE

subset_null_values = df_product_clean.iloc[:, 1:5]
null_rows = df_product_clean[subset_null_values.isnull().all(axis=1)]
print(null_rows.info())

<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   CATEGORY_1    0 non-null      object
 1   CATEGORY_2    0 non-null      object
 2   CATEGORY_3    0 non-null      object
 3   CATEGORY_4    0 non-null      object
 4   MANUFACTURER  0 non-null      object
 5   BRAND         0 non-null      object
 6   BARCODE       0 non-null      object
dtypes: object(7)
memory usage: 0.0+ bytes
None


In [ ]:
# Checking for duplicated values on BARCODE, which is not allowed to connect with transactions
print(df_product_clean['BARCODE'].count())
print(df_product_clean['BARCODE'].nunique())

# Checking what are the duplicated items for BARCODE
duplicated_items = df_product_clean[df_product_clean.duplicated(subset='BARCODE', keep=False)]
duplicated_items = duplicated_items.sort_values(by='BARCODE')
print(duplicated_items)

# Dropping duplicated values por BARCODE
df_product_clean = df_product_clean.drop_duplicates(subset='BARCODE', keep='first')
print(df_product_clean['BARCODE'].count())
print(df_product_clean['BARCODE'].nunique())

# No rows fully null. So I think we are good to go.

841527
841342
               CATEGORY_1    CATEGORY_2                   CATEGORY_3  \
428256  Health & Wellness     Skin Care  Facial Lotion & Moisturizer   
123194  Health & Wellness     Skin Care       Lip Balms & Treatments   
284352             Snacks         Candy             Confection Candy   
813383             Snacks         Candy             Confection Candy   
434164             Snacks      Pretzels             Covered Pretzels   
...                   ...           ...                          ...   
368833             Snacks  Dips & Salsa                       Hummus   
14607              Snacks         Chips                       Crisps   
87568              Snacks         Chips                       Crisps   
171015             Snacks  Nuts & Seeds                 Covered Nuts   
681268             Snacks  Nuts & Seeds                      Almonds   

                      CATEGORY_4                MANUFACTURER  \
428256                       NaN    R.M. PALMER COMPANY, 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Now we'll save the final CSV's to move forward on another notebook, using
# DuckDB
df_user_clean.to_csv('/content/drive/MyDrive/FETCH/user_clean.csv', index=False)
df_transaction_clean.to_csv('/content/drive/MyDrive/FETCH/transaction_clean.csv', index=False)
df_product_clean.to_csv('/content/drive/MyDrive/FETCH/product_clean.csv', index=False)
